# GC 3

## Muhammad Fahrur Rozi Butar-Butar
## FTDS-016

In [1]:
import pandas as pd
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [2]:
  from google.cloud import bigquery

In [3]:
project_id = "emerald-surface-363009" 

In [4]:
client = bigquery.Client(project=project_id)

 #### Identifikasi Data

In [5]:
df = client.query('''
SELECT *
FROM `bigquery-public-data.thelook_ecommerce.orders` 
''').to_dataframe()

In [6]:
df

,order_id,user_id,status,gender,created_at,returned_at,shipped_at,delivered_at,num_of_item
0,6,4,Shipped,F,2022-10-16 14:16:00+00:00,NaT,2022-10-18 01:50:00+00:00,NaT,2
1,20,14,Shipped,F,2021-07-23 09:49:00+00:00,NaT,2021-07-24 14:38:00+00:00,NaT,2
2,51,37,Shipped,F,2021-12-27 02:38:00+00:00,NaT,2021-12-27 15:18:00+00:00,NaT,1
3,52,38,Shipped,F,2022-06-19 15:14:00+00:00,NaT,2022-06-21 12:06:00+00:00,NaT,1
4,56,39,Shipped,F,2019-07-01 15:49:00+00:00,NaT,2019-07-04 10:52:00+00:00,NaT,2
...,...,...,...,...,...,...,...,...,...
125241,125197,99951,Processing,M,2020-03-17 09:05:00+00:00,NaT,NaT,NaT,1
125242,125201,99955,Processing,M,2022-09-26 01:26:00+00:00,NaT,NaT,NaT,4
125243,125204,99960,Processing,M,2020-11-26 17:42:00+00:00,NaT,NaT,NaT,1
125244,125238,99994,Processing,M,2019-11-13 04:52:00+00:00,NaT,NaT,NaT,3


## Poin penjabaran

 #### 1. Berapa jumlah transaksi yang berstatus Complete tiap bulan selama Q1 sampai Q3 di tahun 2022? Insight apa yang bisa kamu berikan?

In [12]:
client.query('''
SELECT
  extract(year from created_at) as year
  , extract(month from created_at) as month
  , COUNT(order_id) as total_sales
FROM `bigquery-public-data.thelook_ecommerce.order_items`
WHERE status = "Complete" and created_at between "2022-01-01" and "2022-09-30"
GROUP BY year, month
ORDER BY year, month
''').to_dataframe()

,year,month,total_sales
0,2022,1,1555
1,2022,2,1458
2,2022,3,1896
3,2022,4,2002
4,2022,5,2142
5,2022,6,2202
6,2022,7,2711
7,2022,8,3146
8,2022,9,3392


Ternyata jumlah transaksi yang berstatus Complete tiap bulan selama Q1 sampai Q3 di tahun 2022 cenderung meningkat

 #### 2. Berikan informasi total penjualan (dalam USD) tiap bulan selama Q1 sampai Q3 di tahun 2022! (Hanya yang transaksi berstatus Complete. Apa informasi yang bisa kamu sampaikan?

In [8]:
client.query('''
SELECT
  extract(year from created_at) as year
  , extract(month from created_at) as month
  , SUM(sale_price) as total_sale_price_USD
FROM `bigquery-public-data.thelook_ecommerce.order_items`
WHERE status = "Complete" and created_at between "2022-01-01" and "2022-09-30"
GROUP BY year, month
ORDER BY year, month
''').to_dataframe()

,year,month,total_sale_price_USD
0,2022,1,92439.170170
1,2022,2,83947.680040
2,2022,3,113608.770018
3,2022,4,121271.810110
4,2022,5,124586.960032
5,2022,6,130889.060151
6,2022,7,164954.250109
7,2022,8,187644.250148
8,2022,9,200580.560122


Ternyata informasi total penjualan (dalam USD) yang berstatus Complete tiap bulan selama Q1 sampai Q3 di tahun 2022 cenderung meningkat

 #### 3. Berapa user yang melakukan transaksi berstatus Complete di tiap bulan dari Q1 sampai Q3 2022? Apa kesimpulanmu?

In [9]:
client.query('''
SELECT
  extract(year from created_at) as year
  , extract(month from created_at) as month
  , COUNT(user_id) as total_user
FROM `bigquery-public-data.thelook_ecommerce.order_items`
WHERE status = "Complete" and created_at between "2022-01-01" and "2022-09-30"
GROUP BY year, month
ORDER BY year, month
''').to_dataframe()

,year,month,total_user
0,2022,1,1555
1,2022,2,1458
2,2022,3,1896
3,2022,4,2002
4,2022,5,2142
5,2022,6,2202
6,2022,7,2711
7,2022,8,3146
8,2022,9,3392


user yang melakukan transaksi berstatus Complete di tiap bulan dari Q1 sampai Q3 2022 cenderung meningkat

 #### 4. Kategori produk apa saja yang paling banyak dibeli (staus transaksi: Complete) di tiap bulannya selama Q1 sampai Q3 tahun 2022? Berikan insight!

In [10]:
client.query('''
SELECT
  year
  , month
  , product_category
  , total_sold
FROM
  ((SELECT
    year
    ,month
    ,product_category
    ,total_sold
    ,DENSE_RANK() OVER (PARTITION BY year, month ORDER BY total_sold DESC) as dense_ranking
  FROM (
  SELECT
    extract(year from oi.created_at) as year
    , extract(month from oi.created_at) as month
    ,   ii.product_category as product_category
    , COUNT(oi.product_id) as total_sold
  FROM `bigquery-public-data.thelook_ecommerce.inventory_items` as ii 
  JOIN `bigquery-public-data.thelook_ecommerce.order_items` as oi
  ON ii.product_id = oi.product_id
  WHERE oi.status = "Complete" AND oi.created_at BETWEEN "2022-01-01" AND "2022-09-30"
  GROUP BY year, month, product_category
  ORDER BY month, total_sold DESC
)))
WHERE dense_ranking = 1
ORDER BY month
''').to_dataframe()

,year,month,product_category,total_sold
0,2022,1,Jeans,2221
1,2022,2,Intimates,2312
2,2022,3,Jeans,2747
3,2022,4,Shorts,2854
4,2022,5,Jeans,3039
5,2022,6,Intimates,3424
6,2022,7,Jeans,4061
7,2022,8,Jeans,4331
8,2022,9,Jeans,4854


Total pemebelian produk yg paling banyak meningkat setiap bulan selama Q1 sampai Q3

 #### 5. Kategori produk apa saja yang paling banyak mendapatkan pendapatan (staus transaksi: Complete) di tiap bulannya selama Q1 sampai Q3 tahun 2022? Berikan insight!

In [11]:
client.query('''
SELECT
  year
  , month
  , product_category
  , total_sale_price_USD
FROM
  ((SELECT
    year
    ,month
    ,product_category
    ,total_sale_price_USD
    ,DENSE_RANK() OVER (PARTITION BY year, month ORDER BY total_sale_price_USD DESC) as dense_ranking
  FROM (
  SELECT
    extract(year from oi.created_at) as year
    , extract(month from oi.created_at) as month
    ,   ii.product_category as product_category
    , SUM(oi.sale_price) as total_sale_price_USD
  FROM bigquery-public-data.thelook_ecommerce.inventory_items as ii 
  JOIN bigquery-public-data.thelook_ecommerce.order_items as oi
  ON ii.product_id = oi.product_id
  WHERE oi.status = "Complete" AND oi.created_at BETWEEN "2022-01-01" AND "2022-09-30"
  GROUP BY year, month, product_category
  ORDER BY month, total_sale_price_USD DESC
)))
WHERE dense_ranking = 1
ORDER BY month
''').to_dataframe()

,year,month,product_category,total_sale_price_USD
0,2022,1,Outerwear & Coats,215697.490669
1,2022,2,Outerwear & Coats,227229.749109
2,2022,3,Jeans,283307.760147
3,2022,4,Outerwear & Coats,283528.310406
4,2022,5,Jeans,317869.810249
5,2022,6,Jeans,313301.300201
6,2022,7,Outerwear & Coats,495873.729740
7,2022,8,Outerwear & Coats,453461.458815
8,2022,9,Outerwear & Coats,547806.789011


Dapat dilihat bahwa total pendapatan meningkat setiap bulannya.

 #### 6. Dibandingkan dengan jumlah transaksi dan total penjualan, mana yang paling berkaitan dengan jumlah user yang melakukan transaksi? Apa analisis yang dapat kamu jelaskan? (Hint: Kamu bisa menggunakan korelasi)

Dengan SQL

In [15]:
client.query('''
SELECT
  CORR(total_user, total_transaction) as corr_user_transaction
  , CORR(total_user, total_sales) as corr_user_sales
FROM(
SELECT 
  EXTRACT(year FROM created_at) AS year
  , EXTRACT(month FROM created_at) AS month
  , COUNT(user_id) as total_user
  , COUNT (order_id) as total_transaction
  , SUM(sale_price) as total_sales
  FROM bigquery-public-data.thelook_ecommerce.order_items
  GROUP BY year,month
  ORDER BY month
)
''').to_dataframe()

,corr_user_transaction,corr_user_sales
0,1.0,0.99988


- 0 : Tidak ada korelasi antara dua variabel
- lebih 0 – 0,25 : Korelasi sangat lemah
- lebih 0,25 – 0,5 : Korelasi cukup
- lebih 0,5 – 0,75 : Korelasi kuat
- lebih 0,75 – 0,99 : Korelasi sangat kuat
- 1 : Korelasi hubungan sempurna positif
- 1 : Korelasi hubungan sempurna negatif

Dari pernyataan diatas maka korelasi antara data sengat kuat

Dengan Pandas

In [18]:
data1 = client.query('''
SELECT 
  total_user
  , total_sales
FROM
(SELECT 
  EXTRACT(year FROM created_at) AS year
  , EXTRACT(month FROM created_at) AS month
  , COUNT(user_id) as total_user
  , SUM(sale_price) as total_sales
  FROM bigquery-public-data.thelook_ecommerce.order_items
GROUP BY year,month
ORDER BY month)
''').to_dataframe()

In [20]:
data1.corr()

,total_user,total_sales
total_user,1.00000,0.99988
total_sales,0.99988,1.00000


In [19]:
data2 = client.query('''
SELECT 
  total_user
  , total_transaction
FROM
(SELECT 
  EXTRACT(year FROM created_at) AS year
  , EXTRACT(month FROM created_at) AS month
  , COUNT(user_id) as total_user
  , COUNT (order_id) as total_transaction
  FROM bigquery-public-data.thelook_ecommerce.order_items
GROUP BY year,month
ORDER BY month)
''').to_dataframe()

In [21]:
data2.corr()

,total_user,total_transaction
total_user,1.0,1.0
total_transaction,1.0,1.0


- 0 : Tidak ada korelasi antara dua variabel
- lebih 0 – 0,25 : Korelasi sangat lemah
- lebih 0,25 – 0,5 : Korelasi cukup
- lebih 0,5 – 0,75 : Korelasi kuat
- lebih 0,75 – 0,99 : Korelasi sangat kuat
- 1 : Korelasi hubungan sempurna positif
- 1 : Korelasi hubungan sempurna negatif

Dari pernyataan diatas maka korelasi antara data sengat kuat

Korelasi transaksi dan user lebih kuat dengan nilai 1.0

#### POIN ANALISIS

Berikan kesimpulan dari laporan/informasi yang dibuat berdasarkan problem statement/poin penjabaran menggunakan bahasa awam! (boleh dalam beberapa paragraf)

- transaksi, total penjualan dan jumlah user meningkat setiap bulan di Q1 - Q3 2022
- Korelasi transaksi dan user lebih kuat dengan nilai 1.0

#### POIN PERTANYAAN

1. Apakah problem statement yang kamu definisikan di awal dapat terukur ketercapaiannya? berikan pendapatmu!

Dengan problem statement yang diatas dapat menggambarkan hubungan-hubungan transaksi, penjualan dan user apakah cenderung meningkat atau tidak. Sehingga memberi gambaran dalam penulisan laporan evaluasi aktivitas penjualan di platform tersebut.

2. Berdasarkan hasil analisis yang sudah kamu lakukan dari 6 penjabaran di atas, jika CEO perusahaanmu ingin menargetkan pendapatan di awal kuartal 4 harus mencapai $250000, apakah masuk akal?

Menurut saya masuk akal, karena penjualan akhir Q3 sebanyak 200580.560122 dan terus meningkat setiap bulan.

3. CEO kamu menargetkan di kuartal 4 ada investor yang dapat menyuntikan dana ke perusahaanmu, dimana investor akan melihat GMV selama 3 kuartal terakhir serta prospek kedepan minimal di kuartal 4 akan seperti apa. Berikan informasi kepada CEO mu berdasarkan trend transaksi, jumlah user yang bertransaksi, dan GMV 3 kuartal terakhir, apakah perusahaanmu layak atau tidak mendapatkan investor baru!

transaksi, total penjualan dan jumlah user meningkat setiap bulan di Q1 - Q3 2022 sehingga perusahaan ini layak untuk diinvest karena pertumbuhannya berjalan dengan baik